In [1]:
!pip install tensorflow opencv-python tqdm psutil scikit-learn

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tqdm import tqdm
import gc
import psutil
import time
import glob
from sklearn.utils import shuffle

print(f"TensorFlow: {tf.__version__}")
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TARGET_NEURONS = 13065
BATCH_SIZE = 32
EPOCHS = 20
IMAGE_SIZE = (48, 48)
DROPOUT_RATE = 0.5
L2_REG = 0.01

print(f"Config: {EPOCHS} epochs, Batch size: {BATCH_SIZE}")

TensorFlow: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Config: 20 epochs, Batch size: 32


In [3]:
print("Downloading dataset...")
start_time = time.time()

!git clone -q https://github.com/chenkenanalytic/handwritting_data_all.git
!cat /content/handwritting_data_all/all_data.zip* > /content/handwritting_data_all/all_data.zip
!unzip -q -O big5 /content/handwritting_data_all/all_data.zip -d "/content"

print(f"Dataset downloaded in {time.time() - start_time:.1f} seconds")

Dataset downloaded in 99.8 seconds


In [4]:
def load_dataset_fast():
    dataset_path = "/content/cleaned_data"

    print("Loading dataset structure...")
    all_folders = os.listdir(dataset_path)

    char_files = {}
    char_count = 0

    # Scan folders for target characters
    for folder in tqdm(all_folders[:2000]):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue

        png_files = glob.glob(os.path.join(folder_path, "*.png"))
        for file_path in png_files:
            char_name = os.path.basename(file_path).split('_')[0]
            if char_name not in char_files:
                if char_count >= TARGET_NEURONS:
                    break
                char_files[char_name] = []
                char_count += 1
            char_files[char_name].append(file_path)

        if char_count >= TARGET_NEURONS:
            break

    print(f"Loaded {len(char_files)} characters")

    # Create character mapping
    char_to_idx = {char: idx for idx, char in enumerate(char_files.keys())}

    return char_files, char_to_idx

char_files, char_map = load_dataset_fast()
print(f"Characters loaded: {len(char_map)}")

Loading dataset structure...


100%|██████████| 2000/2000 [00:00<00:00, 4592.70it/s]

Loaded 2000 characters
Characters loaded: 2000


In [5]:
def load_original_data(char_files, char_map, target_size=(48, 48)):
    print("Loading original dataset...")

    train_images, train_labels = [], []
    test_images, test_labels = [], []

    for char, files in tqdm(char_files.items()):
        files_sorted = sorted(files)

        # Training data (first 40 samples)
        for file_path in files_sorted[:40]:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, target_size)
                img = img.astype('float32') / 255.0
                img = np.expand_dims(img, axis=-1)
                train_images.append(img)
                train_labels.append(char_map[char])

        # Testing data (remaining samples)
        for file_path in files_sorted[40:]:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, target_size)
                img = img.astype('float32') / 255.0
                img = np.expand_dims(img, axis=-1)
                test_images.append(img)
                test_labels.append(char_map[char])

    return (np.array(train_images), np.array(train_labels),
            np.array(test_images), np.array(test_labels))

X_train_orig, y_train_orig, X_test, y_test = load_original_data(char_files, char_map, IMAGE_SIZE)
print(f"Original training data: {X_train_orig.shape}")
print(f"Testing data: {X_test.shape}")

Loading original dataset...


100%|██████████| 2000/2000 [02:07<00:00, 15.68it/s]


Original training data: (79905, 48, 48, 1)
Testing data: (24694, 48, 48, 1)


In [6]:
def optimized_augment_images(images, labels, target_samples_per_class=100):

    print(f"Starting optimized data augmentation - target: {target_samples_per_class} samples per character")

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        fill_mode='constant',
        cval=0
    )

    augmented_images = []
    augmented_labels = []

    unique_labels = np.unique(labels)
    print(f"Processing {len(unique_labels)} unique characters...")

    for class_label in tqdm(unique_labels, desc="Augmentation"):
        class_mask = (labels == class_label)
        class_images = images[class_mask]
        original_count = len(class_images)

        if original_count == 0:
            continue

        augmented_images.extend(class_images)
        augmented_labels.extend([class_label] * original_count)

        needed_augmented = target_samples_per_class - original_count

        if needed_augmented > 0:
            batch_size_aug = min(needed_augmented, 50)

            class_generator = datagen.flow(
                class_images,
                batch_size=batch_size_aug,
                shuffle=False
            )

            aug_batch = next(class_generator)
            num_to_take = min(len(aug_batch), needed_augmented)

            augmented_images.extend(aug_batch[:num_to_take])
            augmented_labels.extend([class_label] * num_to_take)

    X_augmented = np.array(augmented_images)
    y_augmented = np.array(augmented_labels)

    X_augmented, y_augmented = shuffle(X_augmented, y_augmented, random_state=42)

    print(f"Augmentation completed:")
    print(f"  Original samples: {len(images)}")
    print(f"  Augmented samples: {len(X_augmented)}")
    print(f"  Average per class: {len(X_augmented) // len(unique_labels)}")

    return X_augmented, y_augmented

X_train_augmented, y_train_augmented = optimized_augment_images(
    X_train_orig, y_train_orig, target_samples_per_class=100
)

print(f"Final training data: {X_train_augmented.shape}")
print(f"Final training labels: {y_train_augmented.shape}")

Starting optimized data augmentation - target: 100 samples per character
Processing 2000 unique characters...


Augmentation: 100%|██████████| 2000/2000 [00:34<00:00, 57.87it/s]


Augmentation completed:
  Original samples: 79905
  Augmented samples: 159810
  Average per class: 79
Final training data: (159810, 48, 48, 1)
Final training labels: (159810,)


In [7]:
class AdvancedEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=7, min_delta=0.002):
        super().__init__()
        self.patience = patience
        self.min_delta = min_delta
        self.best_weights = None
        self.best_acc = 0
        self.wait = 0
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        current_acc = logs.get('val_accuracy', 0)

        if current_acc > self.best_acc + self.min_delta:
            self.best_acc = current_acc
            self.best_weights = self.model.get_weights()
            self.wait = 0
            print(f"✅ New best validation accuracy: {current_acc:.4f}")
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)
                print(f"🛑 Early stopping triggered at epoch {epoch + 1}")
                print(f"🎯 Restored best weights, validation accuracy: {self.best_acc:.4f}")

def smoothed_sparse_categorical_crossentropy(y_true, y_pred, smoothing=0.1):
    y_true_smooth = y_true * (1 - smoothing) + smoothing / TARGET_NEURONS
    return tf.keras.losses.sparse_categorical_crossentropy(y_true_smooth, y_pred)

In [8]:
def create_simple_effective_model1():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (5, 5), activation='relu',
                              input_shape=(48, 48, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Dense(TARGET_NEURONS, activation='softmax')
    ])
    return model

print("Creating Model 1 (Simple & Effective)...")
model1 = create_simple_effective_model1()

model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model 1 parameters: {model1.count_params():,}")
model1.summary()

Creating Model 1 (Simple & Effective)...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model 1 parameters: 1,779,465


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 44, 44, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 44, 44, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 20, 20, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 20, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 13065)          │     1,685,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,779,465 (6.79 MB)

 Trainable params: 1,779,017 (6.79 MB)

 Non-trainable params: 448 (1.75 KB)

In [9]:
print("=== TRAINING MODEL 1 (Regularized CNN) ===")

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class CleanProgressCallback(tf.keras.callbacks.Callback):
    def __init__(self, total_epochs):
        self.total_epochs = total_epochs

    def on_epoch_end(self, epoch, logs=None):
        try:
            lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        except:
            lr = 0.001

        print(f"Epoch {epoch+1}/{self.total_epochs}: "
              f"accuracy: {logs['accuracy']:.4f} - loss: {logs['loss']:.4f} - "
              f"val_accuracy: {logs['val_accuracy']:.4f} - val_loss: {logs['val_loss']:.4f} - "
              f"learning_rate: {lr:.2e}")

callbacks1 = [
    CleanProgressCallback(total_epochs=15),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=2,
        min_lr=0.00001,
        mode='max',
        verbose=0
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model1.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=0
    )
]

try:
    history1 = model1.fit(
        X_train_augmented, y_train_augmented,
        batch_size=64,
        epochs=15,
        validation_data=(X_test, y_test),
        callbacks=callbacks1,
        verbose=0,
        shuffle=True
    )

    # Load best model and evaluate
    model1 = tf.keras.models.load_model('best_model1.keras')
    test_loss1, test_accuracy1 = model1.evaluate(X_test, y_test, verbose=0)

    print(f"Model 1 Test Accuracy: {test_accuracy1:.4f}")

    model1_filename = f'model1_regularized_{int(time.time())}.keras'
    model1.save(model1_filename)
    print(f"Model 1 saved as {model1_filename}")

except Exception as e:
    print(f"Model 1 training failed: {e}")
    print("Using fallback model...")

gc.collect()

=== TRAINING MODEL 1 (Regularized CNN) ===
Epoch 1/15: accuracy: 0.0222 - loss: 6.6287 - val_accuracy: 0.0136 - val_loss: 8.4215 - learning_rate: 1.00e-03
Epoch 2/15: accuracy: 0.1649 - loss: 4.3808 - val_accuracy: 0.0532 - val_loss: 7.0002 - learning_rate: 1.00e-03
Epoch 3/15: accuracy: 0.3090 - loss: 3.2979 - val_accuracy: 0.0516 - val_loss: 9.5358 - learning_rate: 1.00e-03
Epoch 4/15: accuracy: 0.3999 - loss: 2.7521 - val_accuracy: 0.0901 - val_loss: 9.7219 - learning_rate: 1.00e-03
Epoch 5/15: accuracy: 0.4544 - loss: 2.4251 - val_accuracy: 0.0488 - val_loss: 12.9421 - learning_rate: 1.00e-03
Epoch 6/15: accuracy: 0.4924 - loss: 2.2208 - val_accuracy: 0.2173 - val_loss: 4.2534 - learning_rate: 1.00e-03
Epoch 7/15: accuracy: 0.5210 - loss: 2.0607 - val_accuracy: 0.1474 - val_loss: 5.6087 - learning_rate: 1.00e-03
Epoch 8/15: accuracy: 0.5386 - loss: 1.9612 - val_accuracy: 0.1773 - val_loss: 7.6869 - learning_rate: 1.00e-03
Epoch 9/15: accuracy: 0.5784 - loss: 1.7807 - val_accuracy: 

907

In [10]:
def create_simplified_model2():
    model = tf.keras.Sequential([
        # Simplified but effective architecture
        tf.keras.layers.Conv2D(48, (3, 3), activation='relu',
                              kernel_initializer='he_normal',
                              input_shape=(48, 48, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(96, (3, 3), activation='relu',
                              kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Conv2D(192, (3, 3), activation='relu',
                              kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),

        # Direct to output
        tf.keras.layers.Dense(TARGET_NEURONS, activation='softmax',
                             kernel_initializer='he_normal')
    ])
    return model

print("Creating Model 2 (Simplified Architecture)...")
model2 = create_simplified_model2()

optimizer2 = tf.keras.optimizers.Adam(
    learning_rate=0.0003,
    clipnorm=1.0
)

model2.compile(
    optimizer=optimizer2,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model 2 output neurons: {model2.layers[-1].units}")
print(f"Model 2 parameters: {model2.count_params():,}")

Creating Model 2 (Simplified Architecture)...
Model 2 output neurons: 13065
Model 2 parameters: 2,731,017


In [11]:
print("=== TRAINING MODEL 2 (Simplified Architecture) ===")

callbacks2 = [
    CleanProgressCallback(total_epochs=12),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=2,
        min_lr=0.00001,
        mode='max',
        verbose=0
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model2.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=0
    )
]

try:
    # Train Model 2
    history2 = model2.fit(
        X_train_augmented, y_train_augmented,
        batch_size=64,
        epochs=12,
        validation_data=(X_test, y_test),
        callbacks=callbacks2,
        verbose=0,
        shuffle=True
    )

    # Load best model and evaluate
    model2 = tf.keras.models.load_model('best_model2.keras')
    test_loss2, test_accuracy2 = model2.evaluate(X_test, y_test, verbose=0)

    print(f"Model 2 Test Accuracy: {test_accuracy2:.4f}")

    model2_filename = f'model2_simplified_{int(time.time())}.keras'
    model2.save(model2_filename)
    print(f"Model 2 saved as {model2_filename}")

except Exception as e:
    print(f"Model 2 training failed: {e}")

gc.collect()

=== TRAINING MODEL 2 (Simplified Architecture) ===
Epoch 1/12: accuracy: 0.0021 - loss: 7.7991 - val_accuracy: 0.0111 - val_loss: 6.8045 - learning_rate: 3.00e-04
Epoch 2/12: accuracy: 0.0126 - loss: 6.6194 - val_accuracy: 0.0586 - val_loss: 5.8123 - learning_rate: 3.00e-04
Epoch 3/12: accuracy: 0.0397 - loss: 5.8798 - val_accuracy: 0.1287 - val_loss: 5.0213 - learning_rate: 3.00e-04
Epoch 4/12: accuracy: 0.0822 - loss: 5.2291 - val_accuracy: 0.1319 - val_loss: 4.8247 - learning_rate: 3.00e-04
Epoch 5/12: accuracy: 0.1363 - loss: 4.6608 - val_accuracy: 0.3172 - val_loss: 3.6361 - learning_rate: 3.00e-04
Epoch 6/12: accuracy: 0.1890 - loss: 4.1819 - val_accuracy: 0.2474 - val_loss: 3.9626 - learning_rate: 3.00e-04
Epoch 7/12: accuracy: 0.2459 - loss: 3.7652 - val_accuracy: 0.1885 - val_loss: 5.4815 - learning_rate: 3.00e-04
Epoch 8/12: accuracy: 0.2913 - loss: 3.4652 - val_accuracy: 0.5271 - val_loss: 2.3579 - learning_rate: 1.50e-04
Epoch 9/12: accuracy: 0.3186 - loss: 3.2980 - val_acc

1204

In [12]:
def create_lightweight_model3():
    model = tf.keras.Sequential([
        # Lightweight but strongly regularized
        tf.keras.layers.Conv2D(24, (3, 3), activation='relu',
                              kernel_initializer='he_normal',
                              kernel_regularizer=tf.keras.regularizers.l2(L2_REG),
                              input_shape=(48, 48, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(48, (3, 3), activation='relu',
                              kernel_initializer='he_normal',
                              kernel_regularizer=tf.keras.regularizers.l2(L2_REG)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Conv2D(96, (3, 3), activation='relu',
                              kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),

        # Minimal classification head
        tf.keras.layers.Dense(128, activation='relu',
                             kernel_initializer='he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(TARGET_NEURONS, activation='softmax')
    ])
    return model

print("Creating Model 3 (Lightweight)...")
model3 = create_lightweight_model3()

optimizer3 = tf.keras.optimizers.Adam(
    learning_rate=0.0004
)

model3.compile(
    optimizer=optimizer3,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model 3 output neurons: {model3.layers[-1].units}")
print(f"Model 3 parameters: {model3.count_params():,}")

Creating Model 3 (Lightweight)...
Model 3 output neurons: 13065
Model 3 parameters: 1,751,209


In [13]:
# Cell 12: Model 3 Training with Clean Output
print("=== TRAINING MODEL 3 (Lightweight) ===")

callbacks3 = [
    CleanProgressCallback(total_epochs=10),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=2,
        min_lr=0.00001,
        mode='max',
        verbose=0
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model3.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=0
    )
]

try:
    # Train Model 3
    history3 = model3.fit(
        X_train_augmented, y_train_augmented,
        batch_size=64,
        epochs=10,
        validation_data=(X_test, y_test),
        callbacks=callbacks3,
        verbose=0,
        shuffle=True
    )

    # Load best model and evaluate
    model3 = tf.keras.models.load_model('best_model3.keras')
    test_loss3, test_accuracy3 = model3.evaluate(X_test, y_test, verbose=0)

    print(f"Model 3 Test Accuracy: {test_accuracy3:.4f}")

    model3_filename = f'model3_lightweight_{int(time.time())}.keras'
    model3.save(model3_filename)
    print(f"Model 3 saved as {model3_filename}")

except Exception as e:
    print(f"Model 3 training failed: {e}")

gc.collect()

=== TRAINING MODEL 3 (Lightweight) ===
Epoch 1/10: accuracy: 0.0038 - loss: 7.6990 - val_accuracy: 0.0046 - val_loss: 7.0860 - learning_rate: 4.00e-04
Epoch 2/10: accuracy: 0.0216 - loss: 6.1305 - val_accuracy: 0.0146 - val_loss: 8.1994 - learning_rate: 4.00e-04
Epoch 3/10: accuracy: 0.0482 - loss: 5.4746 - val_accuracy: 0.0896 - val_loss: 4.9777 - learning_rate: 4.00e-04
Epoch 4/10: accuracy: 0.0744 - loss: 5.0703 - val_accuracy: 0.1639 - val_loss: 4.2820 - learning_rate: 4.00e-04
Epoch 5/10: accuracy: 0.0979 - loss: 4.7959 - val_accuracy: 0.0841 - val_loss: 5.8071 - learning_rate: 4.00e-04
Epoch 6/10: accuracy: 0.1194 - loss: 4.6033 - val_accuracy: 0.0961 - val_loss: 5.0492 - learning_rate: 4.00e-04
Epoch 7/10: accuracy: 0.1437 - loss: 4.3909 - val_accuracy: 0.3768 - val_loss: 3.0694 - learning_rate: 2.00e-04
Epoch 8/10: accuracy: 0.1556 - loss: 4.2835 - val_accuracy: 0.1149 - val_loss: 5.5572 - learning_rate: 2.00e-04
Epoch 9/10: accuracy: 0.1638 - loss: 4.2121 - val_accuracy: 0.483

1328

In [14]:
print("=== FINAL RESULTS COMPARISON ===")

results = {}
model_files = {}

try:
    if 'test_accuracy1' in locals():
        results['Model 1 (Regularized CNN)'] = test_accuracy1
        model_files['Model 1 (Regularized CNN)'] = 'best_model1.keras'
    if 'test_accuracy2' in locals():
        results['Model 2 (Simplified)'] = test_accuracy2
        model_files['Model 2 (Simplified)'] = 'best_model2.keras'
    if 'test_accuracy3' in locals():
        results['Model 3 (Lightweight)'] = test_accuracy3
        model_files['Model 3 (Lightweight)'] = 'best_model3.keras'
except:
    pass

if results:
    print("\n📊 Accuracy Results:")
    for name, accuracy in results.items():
        print(f"{name}: {accuracy:.4f}")

    best_model_name = max(results, key=results.get)
    best_accuracy = results[best_model_name]

    print(f"\n🏆 Best Model: {best_model_name}")
    print(f"Best Accuracy: {best_accuracy:.4f}")

    # Select and save best model
    try:
        best_model_file = model_files[best_model_name]
        best_model = tf.keras.models.load_model(best_model_file)

        final_filename = f'best_chinese_character_model_{int(time.time())}.keras'
        best_model.save(final_filename)
        print(f"Best model saved as '{final_filename}'")

        # Verify output neurons
        final_neurons = best_model.layers[-1].units
        print(f"\n✅ Final Verification:")
        print(f"Output neurons: {final_neurons}")
        print(f"Required: {TARGET_NEURONS}")
        print(f"Status: {'✅ CORRECT' if final_neurons == TARGET_NEURONS else '❌ INCORRECT'}")

    except Exception as e:
        print(f"Error saving best model: {e}")

=== FINAL RESULTS COMPARISON ===

📊 Accuracy Results:
Model 1 (Regularized CNN): 0.6941
Model 2 (Simplified): 0.6364
Model 3 (Lightweight): 0.4831

🏆 Best Model: Model 1 (Regularized CNN)
Best Accuracy: 0.6941
Best model saved as 'best_chinese_character_model_1763303853.keras'

✅ Final Verification:
Output neurons: 13065
Required: 13065
Status: ✅ CORRECT
